<a href="https://colab.research.google.com/github/lucia-ferrer/Redes-1/blob/pm/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importar colab - **github**

In [14]:
!git clone https://github.com/lucia-ferrer/Redes-1.git

Cloning into 'Redes-1'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 58 (delta 20), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [15]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 49.3 MB/s 


# IMPORTAR


In [45]:
""" Regression problem: MLP con Keras """
import tensorflow as tf
import numpy as np
import pandas as pd
import keras_tuner as kt #Librería de Keras Tuner

#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout

# CARGAR DATOS compactiv
# Cambiar parámetros correspondientes si el delimitador no es una coma, o si el archivo contiene cabeceras
# pueden ser numpy arrays o dataframes en este caso probamos con ndarrays

train_set = pd.read_csv('/content/Redes-1/training_set.csv', header='infer', delimiter=',')
valid_set = pd.read_csv('/content/Redes-1/validation_set.csv', header='infer', delimiter=',')
test_set = pd.read_csv('/content/Redes-1/validation_set.csv', header='infer', delimiter=',')

# SELECCION DE LA SALIDA. Num de columna del target.
y_train = np.array(train_set.iloc[:,-1:])
x_train = np.array(train_set.iloc[: , :-1])
y_valid = np.array(valid_set.iloc[:,-1:])
x_valid = np.array(valid_set.iloc[: , :-1])
y_test = np.array(test_set.iloc[:,-1:])
X_test = np.array(test_set.iloc[: , :-1])
num_train_samples=len(y_train)


# MODELOS PROFE


In [ ]:
# COMPROBAR DIMENSIONES DE LOS DATOS
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print('X_valid: ',X_valid.shape)
print('y_valid: ',y_valid.shape)
print('X_test: ',X_test.shape)
print('y_test: ',y_test.shape)

In [ ]:
# Para la entrada de la red
input_shape=(X_train.shape [1],) #crea una tupla (21,)

#funciones con diferentes modelos
def create_PM_sigmoid(num_hidden_neurons = 50):
  #1 capa oculta y 1 neurona de salida con sigmoide
  model = Sequential()
  model.add(Dense(num_hidden_neurons, input_shape=input_shape, activation='sigmoid'))
  model.add(Dense(1,activation='sigmoid'))
  return model

def create_PM_relu(num_hidden_neurons = 50):
  #1 capa oculta con relu y 1 neurona de salida lineal
  model = Sequential()
  model.add(Dense(num_hidden_neurons, input_shape=input_shape, activation='relu'))
  model.add(Dense(1,activation='linear'))
  return model

#modelo lineal, solo para comparar con el programa desarrollado.
def create_lineal():
  # Una sola neurona lineal (Adaline)
  model = Sequential()
  model.add(Dense(1, input_shape=input_shape, activation='linear'))
  return model


In [ ]:
#Seleccionar el modelo llamando a la función correspondiente
model=create_PM_relu(20) # 1 capa relu con 20 neuronas y salida lineal
#model=create_PM_sigmoid(20) # 1 capa sigmoid con 20 neuronas y salida sigmoid
#model=create_lineal() # 1 capa salida LINEAL (ADALINE)
model.summary() #visualizar la estructura del modelo


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 20)                440       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 461
Trainable params: 461
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
lr = 0.2
epochs = 300
batch_size=32 #no cambiar este valor.
#Para poder ver la curva de validación hay que poner validation_freq=1. Tarda más
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0), metrics=['mse'] )
historico = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(X_valid,y_valid),
shuffle=False, validation_freq=1)


In [ ]:
#______________________________________________________________________________
## plot de evolución de loss (mse)
from matplotlib import pyplot as plt
plt.plot(historico.history['loss'])
plt.plot(historico.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

#______________________________________________________________________________
#Evaluar el modelo con el fich de test
results_test = model.evaluate(X_test, y_test)
results_test[0]
#resultado es una lista con los valores de loss y las métricas elegidas.
#En este caso son loss y mse que coinciden.

# predicciones en test
test_pred = model.predict(X_test)
#crear un ndarray: columna 1 predicciones, columna 2 target
comp = np.append(test_pred, y_test, axis = 1)
comp_df = pd.DataFrame(comp, columns=['prediccion','target'])
#guardar en fichero las salidas de test y target
np.savetxt('salidas_test.txt',comp_df)
#mostrar 10 primeras filas
print(comp_df.iloc[:10,:])

##FICHEROS


In [ ]:
# GUARDAR RESULTADOS EN FICHEROS
# evolución del entrenamiento
# son los datos que se usan para construir los plots
# En este caso, la variable 'historico' contiene los datos del último entrenamiento realizado
np.savetxt('/Redes-1/historico_pm/historicoTrainLoss.txt',historico.history['loss'])
np.savetxt('/Redes-1/historico_pm/historicoValLoss.txt',historico.history['val_loss'])

errores = [historico.history['loss'][-1], historico.history['val_loss'][-1], results_test[0]]
#lista con los errores de entrenamiento, validación y test

#guarda el modelo completo
model.save('/Redes-1/modelos_pm/modelo.h5')
#guarda solo pesos
model.save_weights('/Redes-1/modelos_pm/pesos.h5')


# HIPERPARAMETRO

In [46]:
# Para la entrada de la red
input_shape=(x_train.shape [1],) #crea una tupla (21,)

# ______________________ CREACION CON HIPERPARAMETROS__________________________
def build_model(hp):
  model = Sequential()
  for _ in range(hp.Int("num_layers", 1, 3)):
    model.add( Dense(
        # tune number of neurons
        units = hp.Int("units", min_value=20, max_value=100, step=20),
        input_shape = input_shape, 
        #tune the activation function 
        activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
        )
    )
    # Tune whether to use dropout. --> Used to avoid Overfitting.
    if hp.Boolean('dropout'):
      model.add(Dropout(rate=0.25))

  #Output layer
  model.add(Dense(1, input_shape=input_shape, activation='relu'))
  # Tune lr
  lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  
  model.compile(
      loss='mean_squared_error', 
      optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0), 
      metrics=['mse'] )
  return model

build_model(kt.HyperParameters())

In [47]:
tuner = kt.RandomSearch(
    hypermodel = build_model, 
    objective = 'mse',
    seed = 50,
    max_trials = 3,
    )

tuner.search_space_summary()


Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 100, 'step': 20, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [49]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_valid, y_valid))
best_model = tuner.get_best_models()[0]

Para este ejemplo hemos configurado algunos de los elementos del algoritmo de búsqueda de la siguiente manera:

- Función objetivo (objetive) es la función que utilizaremos para encontrar el mejor modelo.
- Semilla (seed) es un valor numérico que nos permitirá repetir la experimentación en caso de que sea necesario.
- Número de intentos (max_trials) es el número de iteraciones del proceso de búsqueda. Este valor se puede considerar como condición de parada del algoritmo de búsqueda.
- Número de ejecuciones por intento (executions_per_trial) es el número de ejecuciones de cada proceso de entrenamiento.